In [374]:
import sys
sys.path.append('./../../src')

# others shit
from matplotlib import pyplot

# my shit
from lib.stonk_jar import StonkJar

In [375]:
ticker = 'AAPL'
jar = StonkJar(ticker)
historical_pickle_name = "{0}.technical.1.historical.df.pkl".format(ticker)
data = jar.read_pickle_dataframe(historical_pickle_name)

In [376]:
data.describe()

,v,e_a,e_e,r_ss,r_s,r_h,r_b,r_sb
count,3.860305e+06,3.860305e+06,3.860305e+06,3.860305e+06,3.860305e+06,3.860305e+06,3.860305e+06,3.860305e+06
mean,5.000387e+01,1.680000e+00,1.555857e+00,1.537892e-02,1.537892e-02,1.784524e-01,5.083847e-01,2.824051e-01
std,2.857487e+01,1.376677e-14,1.421086e-14,1.149616e-02,1.149616e-02,1.842182e-02,3.311151e-02,6.699199e-03
min,1.000000e+00,1.680000e+00,1.555857e+00,0.000000e+00,0.000000e+00,1.582734e-01,4.520548e-01,2.773973e-01
25%,2.500000e+01,1.680000e+00,1.555857e+00,0.000000e+00,0.000000e+00,1.582734e-01,4.760274e-01,2.773973e-01
50%,5.000000e+01,1.680000e+00,1.555857e+00,2.397260e-02,2.397260e-02,1.746575e-01,5.000000e-01,2.773973e-01
75%,7.500000e+01,1.680000e+00,1.555857e+00,2.397260e-02,2.397260e-02,1.986301e-01,5.503597e-01,2.913669e-01
max,9.900000e+01,1.680000e+00,1.555857e+00,2.397260e-02,2.397260e-02,2.226027e-01,5.503597e-01,2.913669e-01


LTSM/Keras code below jacked shamelessly from https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/

In [377]:
# training params

n_features = 5

# number of samples to use to train in each round
# samples in this case are hourly tickers, so this is a 8hr trading day
n_samples = 8

# output 1 prediction
n_outputs = 1

# 5 days of trading data at a time
batch_size = n_samples * 5

In [378]:
# prepare data for lstm
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

# drop columns we don't want to predict
data.drop(data.columns[[6, 7, 8, 9, 10, 11, 12]], axis=1, inplace=True)
data.index.name = 'ts'
print(data)
values = data.values[:, 1:(n_features)]
# integer encode direction
encoder = LabelEncoder()
# wtf is the 4 about? why 4?
values[:,4] = encoder.fit_transform(values[:,4])
# ensure all data is float
values = values.astype('float32')
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
# frame as supervised learning
reframed = series_to_supervised(scaled, n_samples, n_outputs)
print(reframed.head())

                       ts                           o  \
ts                                                      
0     2020-11-01 05:00:00                     122.631   
1     2020-11-01 05:01:00                  122.385738   
2     2020-11-01 05:02:00                  122.140967   
3     2020-11-01 05:03:00                  122.690601   
4     2020-11-01 05:04:00                  123.242709   
...                   ...                         ...   
38692 2021-02-28 02:52:00  120734955111893189001216.0   
38693 2021-02-28 02:53:00  121278262409896724004864.0   
38694 2021-02-28 02:54:00  121824014590741237989376.0   
38695 2021-02-28 02:55:00  121580366561559760076800.0   
38696 2021-02-28 02:56:00  122127478211086762639360.0   

                                l                           h  \
ts                                                              
0                             123                    122.8155   
1                         122.754                  122.569869  

IndexError: index 4 is out of bounds for axis 1 with size 4

In [ ]:
# split into train and test sets
values = reframed.values
training_pct = 0.75
n_training = int(len(values) * 0.75)
train = values[:n_training, :]
test = values[n_training:, :]
# split into input and outputs
n_obs = n_samples * n_features
train_X, train_y = train[:, :n_obs], train[:, -n_features]
test_X, test_y = test[:, :n_obs], test[:, -n_features]
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_samples, n_features))
test_X = test_X.reshape((test_X.shape[0], n_samples, n_features))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

>We will define the LSTM with 50 neurons in the first hidden layer and 1 neuron in the output layer for predicting pollution. The input shape will be 1 time step with 8 features.
>
>We will use the Mean Absolute Error (MAE) loss function and the efficient Adam version of stochastic gradient descent.
>
>The model will be fit for 50 training epochs with a batch size of 72. Remember that the internal state of the LSTM in Keras is reset at the end of each batch, so an internal state that is a function of a number of days may be helpful (try testing this).
>
>Finally, we keep track of both the training and test loss during training by setting the validation_data argument in the fit() function. At the end of the run both the training and test loss are plotted.

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

# design network
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(n_outputs))
model.compile(loss='mae', optimizer='adam')
# fit network
history = model.fit(train_X, train_y, epochs=50, batch_size=batch_size, validation_data=(test_X, test_y), verbose=2, shuffle=False)
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
from numpy import concatenate
from math import sqrt
from sklearn.metrics import mean_squared_error

# make a prediction
yhat = model.predict(test_X)
test_X_pred = test_X.reshape((test_X.shape[0], n_samples*n_features))
# invert scaling for forecast
inv_yhat = concatenate((yhat, test_X_pred[:, -(n_features-1):]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]
# invert scaling for actual
test_y_pred = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y_pred, test_X_pred[:, -(n_features-1):]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]
# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

In [ ]:
# take the last 
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)